In [ ]:
!pip install mujoco
!pip install -q mediapy

%env MUJOCO_GL=egl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.0 MB/s eta 0:00:00
env: MUJOCO_GL=egl


In [ ]:
import mujoco
import mediapy as media
import numpy as np

import cvxpy as cp

In [ ]:
xml = """
<mujoco>
  <option timestep="0.01"/>
  <worldbody>
    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
    <body>
      <joint name="prism" type="slide" axis="1 0 0"/>
      <geom type="box" size=".4 .2 .1" rgba="0 1 0 1"/>
      <body>
        <joint type="hinge" axis="0 1 0"/>
        <geom type="cylinder" size=".05" fromto="0 0 0 0 0 1" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>

  <actuator>
    <motor joint="prism"/>
  </actuator>
</mujoco>
"""

In [ ]:
# initialization
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

## rendering
framerate = 30
renderer = mujoco.Renderer(model, height=360, width=640)
camera = mujoco.MjvCamera()
camera.distance = 5

# MPC
nx = model.nq + model.nv + model.na
nu = model.nu

N = 100
Q = 1e2 * np.eye(nx)
R = 1e-4 * np.eye(nu)

A = np.empty((nx, nx))
B = np.empty((nx, nu))

mujoco.mjd_transitionFD(model, data, 1e-6, 1, A, B, None, None)

x = cp.Variable((nx, N+1))
u = cp.Variable((nu, N))

x_init = cp.Parameter(nx)

state_cost = sum([cp.quad_form(x[:, i], Q) for i in range(N + 1)])
input_cost = sum([cp.quad_form(u[:, i], R) for i in range(N)])

objective = cp.Minimize(state_cost + input_cost)
constraints = [
    x[:, 0] == x_init,
    x[:, 1:N] == A @ x[:, 0 : N - 1] + B @ u[:, 0 : N - 1],
]

problem = cp.Problem(objective, constraints)

# Simulation
data.qpos = np.array([1, np.pi/8])

frames = []
while data.time < 20:
  x = np.concatenate((data.qpos, data.qvel, data.act))
  x_init.value = x
  problem.solve()
  data.ctrl = u.value[:,0]
  data.qfrc_applied = np.random.uniform(-20,20,2)
  mujoco.mj_step(model, data)

  if len(frames) < data.time * framerate:
    renderer.update_scene(data, camera)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=framerate)

This browser does not support the video tag.
